# <center> Chapydette Song Example </center>
Chapydette is a fast and flexible change-point estimation package. In this notebook we'll take a look at how to use the change-point estimation methods in Chapydette to estimate the locations of change points in the song Hakuna Matata. You can listen to the song here: https://www.youtube.com/watch?v=ITDD4doC6iw

## Loading the raw features

Chapydette's examples folder includes a csv file data/hakuna_matata.pickle that contains pitches, loudness and timbre features generated by Echo Nest. 

In [1]:
import pickle

song_path = 'data/hakuna_matata.pickle'
try:
    song = pickle.load(open(song_path, 'rb'), encoding='latin1')
except:
    song = pickle.load(open(song_path, 'rb'))
song_data = song['data']
song_times = song['times']

Each row represents the values of the feature at one time point. The rows are ordered by time, with the first row representing the features at the start of the song. There are 840 observations and 26 features.

In [2]:
print('Shape of song features:', song_data.shape)
print('Sample of features:', song_data[0:3, :])

Shape of song features: (840, 26)
Sample of features: [[ 5.35000e-01  6.26000e-01  5.82000e-01  4.79000e-01  5.72000e-01
   1.00000e+00  8.15000e-01  4.97000e-01  5.64000e-01  6.35000e-01
   6.90000e-01  4.51000e-01  0.00000e+00  1.71130e+02  9.46900e+00
  -2.84800e+01  5.74910e+01 -5.00670e+01  1.48330e+01  5.35900e+00
  -2.72280e+01  9.73000e-01 -1.06400e+01 -7.22800e+00 -6.00000e+01
  -6.00000e+01]
 [ 1.57000e-01  2.47000e-01  4.06000e-01  1.00000e+00  2.91000e-01
   8.30000e-02  1.20000e-01  1.56000e-01  3.80000e-02  4.30000e-02
   2.49000e-01  1.53000e-01  3.48260e+01  1.04290e+01 -1.18830e+01
   3.11284e+02  7.02560e+01  8.53900e+00 -3.20400e+01  5.50860e+01
  -7.36300e+00 -2.21440e+01 -5.70240e+01  1.02710e+01 -6.00000e+01
  -9.51100e+00]
 [ 4.96000e-01  3.70000e-01  2.60000e-01  2.89000e-01  2.60000e-01
   3.42000e-01  8.16000e-01  1.00000e+00  5.92000e-01  6.54000e-01
   5.71000e-01  2.45000e-01  3.46040e+01  1.24793e+02  7.14490e+01
   1.42850e+01  3.82700e+01  7.28560e+01  3

## Feature generation
Prior to applying the change-point estimation algorithms, we want to generate more appropriate features based on the raw features above. Chapydette provides three options: bag-of-features, VLAD, and Fisher vectors. We'll try bag-of-features. Bag-of-features creates histograms for sliding windows. To do so, it does the following:
1. Standardizes the data and runs PCA
2. Runs k-means to create a "codebook"
3. Finds the nearest centroid to each observation
4. Forms a sliding window across the observations and creates a histogram within each sliding window based on the results of the previous step

To use the code it is not necessary to understand the above steps. It is only necessary to understand the required inputs. The required inputs are:
- data_features: The raw features
- nclusters: The number of bins to use for the histograms
- window_length: The length of the sliding window. This is terms of time if you input a vector for `times` (see below) or in terms of indices otherwise.

The most noteworthy optional inputs with their defaults in parentheses are:
- data_codebook (None): Dataset to use for codebook generation step. If not specified, the code uses the same data for the codebook generation step and the feature generation step. 
- times (None): Times corresponding to each data point. If you want the sliding windows to be based on time rather than index, you need to specify a vector of times, with one time for each observation. For example, for this song we could input the times in the song at which all of the observations were recorded.
- do_pca (True): Whether to perform PCA on the data
- window_overlap (0.2*window_length): Sliding window overlap, in terms of time.            

For our music example, we'll use the following:
- data_features = the raw features we loaded above
- nclusters = 16. In practice we would want to use many songs for the codebook generation. We would set data_codebook to be the concatenation of the raw features from those songs.
- window_length = 10s
- data_codebook = None
- times = the times in the song at which the features were recorded
- do_pca = True
- window_overlap = 0.2*window_length

The output of the feature generation step consists of several things:
- features: The features that you should input into the change-point estimation algorithm
- interval_start_times: Start time of every sliding window  
- interval_end_times: End time of every sliding window
- scaler: The Scikit-Learn scaler used to standardize the data
- pca: The trained PCA function
- centroids: The centroids from the codebook generation step

The latter three outputs will not be relevant to us now. They could be useful if you are processing data in batches and need to be able to generate features in the same way later.

In [3]:
from chapydette import feature_generation
window_length = 5
nclusters = 16
features, interval_start_times, interval_end_times, scaler, pca, centroids = feature_generation.bag_of_features(song_data, nclusters, 
                                                                                            window_length, times=song_times, seed=0)

Running PCA...
Running k-means...
Generating histograms...
Done generating features.


## Change-point estimation
Chapydette currently contains two different kernel-based change-point algorithms. The first one is based on the Maximum Mean Discrpancy (MMD) and computes the location of one change point (Gretton et al. (2012), Harchaoui et al. (2009)). The second is an implementation of the multiple kernel change-point estimation algorithm of Harchaoui and Cappé (2007). Both algorithms may be called from the cp_estimation module.

The potential inputs to both algorithms with their defaults in parentheses are:
- X (None): Matrix of observations. Each observation is one row.
- gram (None): Pre-computed gram matrix
- n_cp (1): Number of change points to estimate
- est_ncp (False): Whether to estimate the number of change points using the method of Arlot et al. (2019).
- kernel_type ('detect'): Type of kernel to use. One of: 'precomputed', 'chi-squared', 'gaussian-euclidean', 'gaussian-tv', 'linear'
- bw (None): Bandwidth for the kernel (if applicable)
- min_dist (1): Minimum allowable distance between successive change points
- alpha (2): Parameter in the method of Arlot et al. (2019) used to estimate the number of change points.
- return_obj (False): Whether to return the optimal objective values.

You must provide either X or gram. If you do not specify kernel_type, the code will try to determine an appropriate kernel to use. If you do not provide a bandwidth, the code will try to use a rule of thumb for the kernel. 

The outputs of the algorithms are the estimated change points, in terms of the indices of the observations. The estimated change points are the indices of the last element in each estimated segment.

Let's first try the MMD-based algorithm. 

In [4]:
from chapydette import cp_estimation
cp = cp_estimation.mmd_cpd(X=features, gram=None, n_cp=1, kernel_type='detect', bw=None, min_dist=1)
print('Estimated last element in the first segment is', cp)

Using the Gaussian kernel with the Hellinger distance.
Bandwidth not specified. Using a rule of thumb.
Estimated last element in the first segment is 43


The algorithm estimated that there is a change point between the cpth and (cp+1)th observations (time intervals). Let's now use the `convert_cps_to_time` function in utils to see what time in the song this change point corresponds to.

In [5]:
from chapydette import utils
cp_times = utils.convert_cps_to_time([cp], interval_start_times, interval_end_times)

print('Estimated change-point time:', cp_times[0], 'seconds')

Estimated change-point time: 174.25135999999998 seconds


I encourage you to listen to the song to verify that there is a change point at approximately this time.

Next, let's try the multiple kernel change-point algorithm. This time we'll search for five change points.

In [6]:
import numpy as np

cps = cp_estimation.mkcpe(X=features, gram=None, n_cp=5, kernel_type='detect', bw=None, min_dist=1)
cp_times = utils.convert_cps_to_time(cps.flatten(), interval_start_times, interval_end_times)

print('Estimated change-point times:', cp_times)

Using the Gaussian kernel with the Hellinger distance.
Bandwidth not specified. Using a rule of thumb.
Estimated change-point times: [42.2722, 102.28687, 138.05104, 174.25135999999998, 202.160275]


## Other options
There are a lot of other things we could've tried, including different feature generation methods and different kernels. Below are two other options. Feel free to try more yourself and see how the results compare.

In [7]:
window_length = 10
nclusters = 8
features, interval_start_times, interval_end_times, scaler, pca, centroids = feature_generation.vlad(song_data, nclusters, 
                                                                                            window_length, times=song_times, seed=0)
cps = cp_estimation.mkcpe(X=features, gram=None, n_cp=5, kernel_type='linear', bw=None, min_dist=1)
cp_times = utils.convert_cps_to_time(cps.flatten(), interval_start_times, interval_end_times)
print(cp_times)

Running PCA...
Running k-means...
Generating VLAD features...
Done generating features.
[43.929024999999996, 84.08796, 100.18517, 139.90345000000002, 188.08968]


In [8]:
window_length = 10
nclusters = 8
features, interval_start_times, interval_end_times, scaler, pca, centroids = feature_generation.fisher_vectors(song_data, nclusters, 
                                                                                            window_length, times=song_times, seed=0)
cps = cp_estimation.mkcpe(X=features, gram=None, n_cp=5, kernel_type='linear', bw=None, min_dist=1)
cp_times = utils.convert_cps_to_time(cps.flatten(), interval_start_times, interval_end_times)
print(cp_times)

Running PCA...
Running GMM...
Generating Fisher vector features...
Done generating features.
[20.14356, 43.929024999999996, 100.18517, 139.90345000000002, 180.06254]


## References:
- Gretton, A., Borgwardt, K. M., Rasch, M. J., Schölkopf, B., & Smola, A. (2012). A kernel two-sample test. Journal of Machine Learning Research, 13(Mar), 723-773.
- Harchaoui, Z., & Cappé, O. (2007, August). Retrospective multiple change-point estimation with kernels. In IEEE/SP 14th Workshop on Statistical Signal Processing, 2007. SSP'07. (pp. 768-772). IEEE.
- Harchaoui, Z., Moulines, E., & Bach, F. R. (2009). Kernel change-point analysis. In Advances in Neural Information Processing Systems (pp. 609-616).